In [8]:
#-*- coding: utf-8 -*-
#

import json
import requests 
from pprint import pprint
from urllib import request
import pandas as pd
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta 

# 외부 인자로 직접 받기
#import argparse
#parser = argparse.ArgumentParser(description='Process some integers.')
#parser.add_argument('token',  type=str, help='api_token')
#parser.add_argument('sheet_url', type = str, help='google_sheet_url')
#parser.add_argument('sheet_name', type = str, help='google_sheet_name')

# 텍스트 파일에 저장 후 읽어들이기
with open('/Users/jinseulpark/Desktop/write/band_automation/cred_info.txt', 'r',encoding= 'utf8') as f:
    lin = f.read()
    lin_list = lin.split('\n')
    
token = lin_list[0] # 네이버 특정 밴드와 연동하기 위한 토큰, 신규 밴드 생성시 해당 값도 수정이 필요함
sheet_url = lin_list[1] # 구글 시트 url
sheet_name = lin_list[2] # 구글 시트명
band_name = lin_list[3] # 네이버 밴드명

In [10]:

def get_bands():
    url = f'https://openapi.band.us/v2.1/bands?access_token={token}'
    req = request.Request(url)
    res = request.urlopen(req)
    decoded = res.read().decode('utf8')
    json_dict = json.loads(decoded)
    return json_dict


In [167]:
def get_nickname_summary(result_df):
    
    result_df['nickname'] = result_df['author'].apply(lambda x : x['name'])
    result_df['w_url'] = result_df['content'].apply(lambda x : re.findall(r'(https?://\S+)', x)[0] if len(re.findall(r'(https?://\S+)', x)) > 0 else 'not_url' )
    result_df['created_date'] = result_df['created_at'].apply(lambda x : pd.to_datetime(datetime.fromtimestamp(x/1000), format = '%Y-%m-%d').date())
    if 'latest_comments' not in  result_df.columns:
        result_df['latest_comments'] = [np.nan] * result_df.shape[0]
    return result_df[['nickname','content','post_key','w_url','created_date','latest_comments']]

In [168]:
def get_post(next_params, do_push = True):
    url = 'https://openapi.band.us/v2/band/posts'
    with requests.post(url, next_params) as resp:
        json_dict = json.loads(resp.text)
        return json_dict
    # 로케일 : 사용자 인터페이스에서 사용되는 언어, 지역설정, 출력형식 등을 정의하는 문자열
    # 한국어의 경우 ko_KR.UTF-8 (language territory codeset 순서)


In [169]:
def create_post(band_key, content, do_push = True):
    url = 'https://openapi.band.us/v2.2/band/post/create'
    params = {'access_token': token, 
              'band_key': band_key,
              'content': content, 'do_push' : do_push}
    with requests.post(url, params) as resp:
        json_dict = json.loads(resp.text)
        return json_dict

In [170]:
def create_comment(band_key, body, post_key):
    url = 'https://openapi.band.us/v2/band/post/comment/create'
    params = {'access_token': token, 
              'band_key': band_key,
              'body': body, 
              'post_key' : post_key}
    with requests.post(url, params) as resp:
        json_dict = json.loads(resp.text)
        return json_dict
    

In [171]:
band_info = [i for i in get_bands()['result_data']['bands'] if i['name']== band_name][0]
next_params = {'access_token': token, 
              'band_key': band_info['band_key'],
              'locale': 'ko_KR'}

df_all_contents = pd.DataFrame()
while next_params != 'stop':
    if next_params is None:
        next_params = 'stop'
        
    else:
        result = get_post(next_params)
        next_params = result['result_data']['paging']['next_params']

        # 데이터 추출
        result_df = pd.DataFrame(result['result_data']['items'])
        df_ = get_nickname_summary(result_df)
        df_all_contents = pd.concat([df_, df_all_contents])
        #print(next_param)
    
print('data extraction is finished')

data extraction is finished


In [174]:
# 1. 저자의 매일 포스트 체크 daily check posting

In [175]:
user_list = df_all_contents.nickname.unique()# 전체 참가자 추출
df_all = df_all_contents[df_all_contents['w_url'] != 'not_url'] # url없는 경우를 제외
df_all_daily = df_all.groupby(['created_date','nickname'])['w_url'].agg(sum).unstack() # url 없는 글을 제외하고 일별 정리
target_date = pd.to_datetime(datetime.today() - relativedelta(days = 1), format = '%Y-%m-%d').date()
#target_date = target_date = pd.to_datetime(datetime.today(),format = '%Y-%m-%d').date()

# 아예 한번도 작성하지 않아 daily 데이터프레임에 없는 경우, 전체 참가자 리스트에서 수정
import numpy as np
if set(df_all_daily.columns) != set(user_list):
    no_user = [i for i in (set(user_list) - set(df_all_daily.columns))]
    for n in no_user:
        df_all_daily[f'{n}'] = [np.nan] * df_all_daily.shape[0]
        
r = df_all_daily.loc[[target_date]].isnull().T

# 중간 종료자 체크
if r.shape[0] > 0:
    get_late_name = [i for i in r[r[target_date]== True].index]
    #get_late_name.remove('닉네임')
    
else:
    get_late_name = []

from datetime import datetime
target_d = datetime.strftime(target_date, format= '%m/%d')
if len(get_late_name) >0 :
    target_user = ' '.join([f'#{i}' for i in get_late_name])
    content = f'#{target_d} {target_user} 지각입니다! \n 보충글url 은 댓글로 달아주세요:)'
else:
    content = f'#{target_d} 우리 모두 각자의 100일 글쓰기를 완료한 날입니다. 멋져요!'

In [176]:
#status_code = create_post(band_info['band_key'], content)['result_code']
#print(status_code)
print(content)

#05/06 #지하철퇴근 지각입니다! 
 보충글url 은 댓글로 달아주세요:)


In [195]:
from dateutil.relativedelta import relativedelta
post_comment_dict = {}
last_post_id = {}

for row in df_all_contents.values:
    content = row[1]
    post_id = row[2]
    post_created_at= row[4] - relativedelta(days=1)
    post_created_at_str = post_created_at.strftime('%Y-%m-%d')
    if '우리 모두 각자의' in content:
        #print(post_check_at)
        if post_created_at not in post_comment_dict:
            post_comment_dict[post_created_at_str] = {'content': content}
        for user in df_all_daily.columns:
            post_comment_dict[post_created_at_str][user+'_late'] = 'GOOD'
            
    elif '보충글url 은 댓글로 달아주세요' in content:
            content_keywords = [i for i in content.split() if i.startswith('#')]
            #late_date = '2023/' + content_keywords[0].replace('#','')
            #late_date = late_date.replace('/','-')
            late_user = content_keywords[1:]
            #print(late_user)
            if post_created_at not in post_comment_dict:
                post_comment_dict[post_created_at_str] = {'content': content}
            if post_created_at not in last_post_id:
                last_post_id[post_created_at_str] = {'post_key': post_id}
            for user in df_all_daily.columns:
                if '#'+user in late_user:
                    check_at =  datetime.today()
                    delay_duration = post_check_at - check_at
                    #print(delay_duration)
                    post_comment_dict[post_created_at_str][user+'_late'] =  delay_duration
                else:
                    post_comment_dict[post_created_at_str][user+'_late'] = 'GOOD'
                
            #print(post_comment_dict)
            # 댓글 정보 가져오기
            
                for comments in row[5]:
                    user = comments['author']['name']
                    comments_url = comments['body']
                    url_pattern = re.compile(r'(https?://\S+)')
                    urls = url_pattern.findall(comments_url)
                    post_comment_dict[post_created_at_str][user+'_late'] = urls[0]

                #print(post_comment_dict)
            else:
                continue

In [197]:
last_url_comment_df =pd.DataFrame(post_comment_dict).T.sort_index()
date_list = [ i for i in post_comment_dict.keys()]

In [200]:
post_comment_result = pd.DataFrame(post_comment_dict[d].items())

In [208]:
import datetime

# 현재까지 진행한 전체 일자에서 지각글 잘 달려있는지 확인
for d in sorted(date_list):
    
    commented_user = [(k, v) for k, v in post_comment_dict[d].items() if (type(v) != str) and (v < datetime.timedelta(days=-7))]
    #print(d, commented_user)
    for c in commented_user:
        nickname = '#' + c[0].replace('_late','')
        late_period = abs((late_period.days))
        if late_period % 7 == 0: # 7일 이상 지각글 보충 안할 경우, 7일에 1번 댓글로 언급
            comment_content = f'{nickname}, 지각글 기다리고 있어요!'
            create_comment(band_info['band_key'], body = comment_content, post_key=last_post_id[d]['post_key'])
            print(comment_content)